# Calculo de $\pi$ secuencial

In [19]:
%%writefile phiseccollab.c

#include <stdlib.h>
#include <stdio.h>
#include <sys/time.h> //For linux
//#include <windows.h> //for windows OS

#define iterations 1e9

void pileibnitz(double *picalc){
    for(int i=0; i< iterations; i++){
        *picalc += (double) 1/(i*2+1) * 4;
        i++;
        *picalc -= (double) 1/(i*2+1) * 4 ;
    }
}

void main(){
    FILE *fo;
    fo = fopen ("pithreaddataCOLLAB.txt", "a");
    double pi=0;
    struct timeval tval_before, tval_after, tval_result; //time execution calc command for linux
    gettimeofday(&tval_before, NULL); //time execution calc command for linux

    //LARGE_INTEGER frequency; //time execution calc command for windows
    //LARGE_INTEGER start; //time execution calc command for windows
    //LARGE_INTEGER end; //time execution calc command for windows
    //double exec_time; //time execution calc command for windows
    //QueryPerformanceFrequency(&frequency); //time execution calc command for windows
    //QueryPerformanceCounter(&start); //time execution calc command for windows

    pileibnitz(&pi);

    //QueryPerformanceCounter(&end); //time execution calc command for windows
    //exec_time = (double) (end.QuadPart - start.QuadPart)/frequency.QuadPart; //time execution calc command for windows

    gettimeofday(&tval_after, NULL); //time execution calc command for linux
    timersub(&tval_after, &tval_before, &tval_result); //time execution calc command for linux
    long int seconds = (long int)tval_result.tv_sec; //time execution calc command for linux
    long int useconds = (long int)tval_result.tv_usec; //time execution calc command for linux

    //fprintf(fo, "1 %f\n", exec_time);  //time execution calc command for windows
    fprintf(fo, "1 %ld.%06ld\n", seconds, useconds); //time execution calc command for linux
    fclose(fo);

    printf("Pi calculated with %f iteractions \n With a value of %f \n Executed in %ld.%06ld seconds", iterations, pi, seconds, useconds); //time execution calc command for linux
    //printf("Pi calculated with %f iteractions \n With a value of %f \n Executed in %f seconds", iterations, pi, exec_time); //time execution calc command for windows
}

Overwriting phiseccollab.c


In [20]:
%%shell 
gcc phiseccollab.c -o output
./output

Pi calculated with 1000000000.000000 iteractions 
 With a value of 3.141593 
 Executed in 10.661031 seconds

# Calculo de $\pi$ Hilos

In [17]:
%%writefile phithcollab.c

#include <stdlib.h>
#include <stdio.h>
#include <sys/time.h> //For linux
//#include <windows.h> //for windows OS
#include <pthread.h>

#define iterations 1e09
#define threads 4 //change the number threads
double pithval[threads];
int pith[threads];
double pi = 0.0;

void *pileibnitz(void *arg){
    int init, ending, div = *(int *)arg;
    init = (iterations/threads) * div;
    ending = init + ((iterations/threads) -1);
    pithval[div] = 0.0;
    //printf("Init %i, Ending %i, div %i \n",init,ending,div);
    for(int i=init; i< ending; i++){
        pithval[div] += (double) (1.0/(i*2+1) * 4.0);
        i++;
        pithval[div] -= (double) (1.0/(i*2+1) * 4.0);
    }
}

void main(){
    FILE *fo;
    fo = fopen ("pithreaddataCOLLAB.txt", "a");
    pthread_t thread[threads];
    int *retval;

    //LARGE_INTEGER frequency; //time execution calc command for windows
    //LARGE_INTEGER start; //time execution calc command for windows
    //LARGE_INTEGER end; //time execution calc command for windows
    //double exec_time; //time execution calc command for windows
    //QueryPerformanceFrequency(&frequency); //time execution calc command for windows
    //QueryPerformanceCounter(&start); //time execution calc command for windows

    struct timeval tval_before, tval_after, tval_result; //time execution calc command for linux
    gettimeofday(&tval_before, NULL); //time execution calc command for linux

    for (int i=0; i< threads; i++){
        pith[i]=i;
        pthread_create(&thread[i], NULL, pileibnitz, &pith[i]);
    }
    for (int j=0; j< threads; j++){
        pthread_join(thread[j], (void **) &retval);
        //printf("\nFinished %i join thread\n", j);
    }
    for (int k=0; k< threads; k++){
        //printf("Pi value in thread %i %2.12f \n", k,pithval[k]);
        pi += pithval[k];
    }


    //QueryPerformanceCounter(&end); //time execution calc command for windows
    //exec_time = (double) (end.QuadPart - start.QuadPart)/frequency.QuadPart; //time execution calc command for windows

    gettimeofday(&tval_after, NULL); //time execution calc command for linux
    timersub(&tval_after, &tval_before, &tval_result); //time execution calc command for linux
    long int seconds = (long int)tval_result.tv_sec; //time execution calc command for linux
    long int useconds = (long int)tval_result.tv_usec; //time execution calc command for linux

    //fprintf(fo, "%i %f\n", threads,exec_time);  //time execution calc command for windows
    fprintf(fo, "%i %ld.%06ld\n", threads,seconds, useconds); //time execution calc command for linux
    fclose(fo);

    printf("Pi calculated with %f iteractions \n With a value of %f \n Executed in %ld.%06ld seconds", iterations, pi, seconds, useconds); //time execution calc command for linux
    //printf("Pi calculated with %f iteractions \n With a value of %f \n Executed in %f seconds", iterations, pi, exec_time); //time execution calc command for windows
}

Overwriting phithcollab.c


In [18]:
%%shell 
gcc phithcollab.c -pthread -o output
./output

Pi calculated with 1000000000.000000 iteractions 
 With a value of 3.141593 
 Executed in 5.146306 seconds

In [ ]:
! lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              2
On-line CPU(s) list: 0,1
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               79
Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:            0
CPU MHz:             2199.998
BogoMIPS:            4399.99
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            56320K
NUMA node0 CPU(s):   0,1
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_sin